FACE RECOGNITION FOR DATA SCIENCE TEAM

In [1]:
#import  required libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dropout,BatchNormalization,Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [2]:
#data generation for training data
train_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.1,horizontal_flip=False,validation_split=0.15)
#train dataset
train_data=train_gen.flow_from_directory(r"C:\Users\hp\Desktop\VS Code\Face_Recognition_Python_Team\Data\Train",
                                          target_size=(224,224),batch_size=32,class_mode="categorical",subset="training")

Found 8984 images belonging to 9 classes.


In [3]:
#classes name
train_data.class_indices

{'arun': 0,
 'chaitanya': 1,
 'laxmi': 2,
 'navin': 3,
 'papu': 4,
 'priti': 5,
 'rahul': 6,
 'sangram': 7,
 'sanket': 8}

In [4]:
#validation dataset
validation_data=train_gen.flow_from_directory(r"C:\Users\hp\Desktop\VS Code\Face_Recognition_Python_Team\Data\Train",
                                          target_size=(224,224),batch_size=32,class_mode="categorical",subset="validation")

Found 1580 images belonging to 9 classes.


In [5]:
#classes name
validation_data.class_indices

{'arun': 0,
 'chaitanya': 1,
 'laxmi': 2,
 'navin': 3,
 'papu': 4,
 'priti': 5,
 'rahul': 6,
 'sangram': 7,
 'sanket': 8}

BUILD CNN ARCHITECTURE USING TRANSFER LEARNING

In [6]:
#define function of CNN for image classification
def cnn_model(activation_function,number_of_classes,optimizer_name):

    #MOBILENETV2
    mob_net=MobileNetV2(weights="imagenet",input_shape=(224,224,3),include_top=False)
    mob_net.trainable=False
    
    #create a sequential model
    model=Sequential()
    #add mobilenet model to sequential model
    model.add(mob_net)
    #convert the model into 1d array
    model.add(Flatten())
    
    #1st fully connnected layer with dropout and batchnormalization
    model.add(Dense(units=64,activation=activation_function))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    
    #2st fully connnected layer with dropout and batchnormalization
    model.add(Dense(units=32,activation=activation_function))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    #3st fully connnected layer with dropout
    model.add(Dense(units=16,activation=activation_function))
    model.add(Dropout(0.25))
    
    #final output layer with softmax activation function
    model.add(Dense(units=number_of_classes,activation="softmax"))
    
    #compile model
    model.compile(optimizer=optimizer_name,loss="categorical_crossentropy",metrics=["accuracy"])
    
    #return our cnn model
    return model

In [7]:
#call our cnn model with relu activation function and adam optimizer
model=cnn_model("relu",9,"adam")
#summmary of our model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 64)                4014144   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 32)                2

In [8]:
#apply early stopping for avoid overfitting
es=EarlyStopping(monitor="accuracy",patience=3)
#train our model
model.fit(train_data,validation_data=validation_data,callbacks=[es],epochs=10)

Epoch 1/10
281/281 [==============================] - 617s 2s/step - loss: 1.0931 - accuracy: 0.6450 - val_loss: 0.3218 - val_accuracy: 0.9335
Epoch 2/10
281/281 [==============================] - 666s 2s/step - loss: 0.3507 - accuracy: 0.9014 - val_loss: 0.1817 - val_accuracy: 0.9418
Epoch 3/10
281/281 [==============================] - 791s 3s/step - loss: 0.2444 - accuracy: 0.9270 - val_loss: 0.1633 - val_accuracy: 0.9544
Epoch 4/10
281/281 [==============================] - 770s 3s/step - loss: 0.1971 - accuracy: 0.9410 - val_loss: 0.1463 - val_accuracy: 0.9582
Epoch 5/10
281/281 [==============================] - 855s 3s/step - loss: 0.1513 - accuracy: 0.9556 - val_loss: 0.1471 - val_accuracy: 0.9639
Epoch 6/10
281/281 [==============================] - 898s 3s/step - loss: 0.1390 - accuracy: 0.9566 - val_loss: 0.1909 - val_accuracy: 0.9456
Epoch 7/10
281/281 [==============================] - 664s 2s/step - loss: 0.1457 - accuracy: 0.9559 - val_loss: 0.1192 - val_accuracy: 0.9747

In [9]:
#save our model in h5 format
model.save("model.h5")